# Structured Outputs

The Augmented LLM

![Augmented LLM](https://www.anthropic.com/_next/image?url=https%3A%2F%2Fwww-cdn.anthropic.com%2Fimages%2F4zrzovbb%2Fwebsite%2Fd3083d3f40bb2b6f477901cc9a240738d3dd1371-2401x1000.png&w=3840&q=75)


Structured Output: Obtain an answer to as a string, but as json format  
Usually, pydantic library is used  


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from mistralai import Mistral
from qdrant_client import QdrantClient

c:\Users\manuelalberto.romero\Documents\repos\dslabs\dslab-rag-retrieval\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data= """
A LA ATT. DE SEGUROS PELAYO:
EN DON BENITO, A 08 MARZO 2014

YO, CARMEN ESPAÑOLA ESPAÑOLA,
CON DNI 99999999R, QUIERO DARME
DE BAJA DEL SEGURO DE COCHE QUE
TENGO CON USTEDES POR LA VENTA DEL
MISMO.

EL NÚMERO DE PÓLIZA ES 0000h2024038
Y SE CORRESPONDE CON UN OPEL CORSA 1.2L
CON MATRÍCULA 5473 BXM

[signature]

Dta. CARMEN ESPAÑOLA
ESPAÑOLA
"""


# Base example with pydantic

In [ ]:
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import Optional



client = OpenAI()

class DatosPoliza(BaseModel):
    fecha: Optional[str] = Field(description="Si existe, la fecha a la que se firma la solicitud")
    dni:  Optional[str] = Field(description="Si existe, el DNI o número de pasarporte del solicitante")
    nro_poliza:  Optional[str]  = Field(description="Si existe, el número de póliza sobre el que el solicitante desea realizar una acción")
    marca_model:  Optional[str] = Field(description="Si aparece, la marca y/o el modelo del vehículo asociado a la póliza")
    matricula:  Optional[str] = Field(description="Si aparece, la matrícula vehículo asociado a la póliza")



response = client.responses.parse(
    model="gpt-4.1-2025-04-14",
    input=[
        {
            "role": "system",
            "content": "Eres un analista que se dedica al cribado de solicitudes de seguros\
                Tu misión es leer detenidamente el correo de un asegurado y extraer información clave",
        },
        {
            "role": "user",
            "content": f"Correo de solicitud: {data}"
        },
    ],
    text_format=DatosPoliza,
)

data_extracted = response.output_parsed

In [15]:
response

ParsedResponse[DatosPoliza](id='resp_68c1a87eb6148193a21cc306a1a789e901849df0f957c36d', created_at=1757522046.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-2025-04-14', object='response', output=[ParsedResponseOutputMessage[DatosPoliza](id='msg_68c1a87fd8a88193bd5c03ae5f7158d901849df0f957c36d', content=[ParsedResponseOutputText[DatosPoliza](annotations=[], text='{"fecha":"08 marzo 2014","dni":"99999999R","accion":"BAJA","nro_poliza":"0000h2024038","marca_model":"OPEL CORSA 1.2L","matricula":"5473 BXM"}', type='output_text', logprobs=[], parsed=DatosPoliza(fecha='08 marzo 2014', dni='99999999R', accion='BAJA', nro_poliza='0000h2024038', marca_model='OPEL CORSA 1.2L', matricula='5473 BXM'))], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, background=False, conversation=None, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=Non

In [14]:
data_extracted



DatosPoliza(fecha='08 marzo 2014', dni='99999999R', accion='BAJA', nro_poliza='0000h2024038', marca_model='OPEL CORSA 1.2L', matricula='5473 BXM')

# Add a predefined set of values

In [16]:
from enum import Enum

class AccionesPoliza(str, Enum):
    BAJA = "BAJA"
    REVISION = "REVISION"
    ANULACION = "ANULACION"
    
    
class DatosPoliza(BaseModel):
    fecha: Optional[str] = Field(description="Si existe, la fecha a la que se firma la solicitud")
    dni:  Optional[str] = Field(description="Si existe, el DNI o número de pasarporte del solicitante")
    accion:  Optional[AccionesPoliza] = Field(description="Si se expresa claramente, la acción que se solicita.")
    nro_poliza:  Optional[str]  = Field(description="Si existe, el número de póliza sobre el que el solicitante desea realizar una acción")
    marca_model:  Optional[str] = Field(description="Si aparece, la marca y/o el modelo del vehículo asociado a la póliza")
    matricula:  Optional[str] = Field(description="Si aparece, la matrícula vehículo asociado a la póliza")



response = client.responses.parse(
    model="gpt-4.1-2025-04-14",
    input=[
        {
            "role": "system",
            "content": "Eres un analista que se dedica al cribado de solicitudes de seguros\
                Tu misión es leer detenidamente el correo de un asegurado y extraer información clave",
        },
        {
            "role": "user",
            "content": f"Correo de solicitud: {data}"
        },
    ],
    text_format=DatosPoliza,
)

data_extracted = response.output_parsed

data_extracted

DatosPoliza(fecha='08 marzo 2014', dni='99999999R', accion=<AccionesPoliza.BAJA: 'BAJA'>, nro_poliza='0000h2024038', marca_model='OPEL CORSA 1.2L', matricula='5473 BXM')

In [ ]:
data_extracted.accion.value

AttributeError: 'Order' object has no attribute 'accion'

# Nesting

In [19]:
from pydantic import BaseModel, PositiveInt, Field
from typing import List

class Product(BaseModel):
    name: str
    quantity: PositiveInt = Field(description="The number of units of this product.")
    unit_price: float = Field(description="The price per unit of the product.")

class Order(BaseModel):
    order_number: str
    items: List[Product]
    customer_name: str
    total_cost: float

order_text = "Order #12345. I purchased two t-shirts at $25 each, and one hoodie for $50. The total came to $100. My name is Jane Doe."


response = client.responses.parse(
    model="gpt-4.1-2025-04-14",
    input=[
        {
            "role": "system",
            "content": "You analize orders recipients. Your task is to extract key information accuratelly"
        },
        {
            "role": "user",
            "content": f"{order_text}"
        },
    ],
    text_format=Order,
)

data_extracted = response.output_parsed

data_extracted

Order(order_number='12345', items=[Product(name='t-shirt', quantity=2, unit_price=25.0), Product(name='hoodie', quantity=1, unit_price=50.0)], customer_name='Jane Doe', total_cost=100.0)

# Other tools

[LangExtract](https://github.com/google/langextract) released by Google in July 30th, is a open source project for large scale, model agnostic (but Gemini first) and example guided information extraction